In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("241211_03_UDF").getOrCreate()
spark

24/12/11 16:08:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
datas = [
    ("A", "2022-04-16", 31200),
    ("B", "2022-04-17", 41200),
    ("C", "2022-04-11", 31500),
    ("D", "2022-04-12", 21500),
    ("E", "2022-04-13", 51000)
]
columns = ["product", "date", "price"]

In [6]:
df = spark.createDataFrame(data =datas, schema = columns)
df.show()

+-------+----------+-----+
|product|      date|price|
+-------+----------+-----+
|      A|2022-04-16|31200|
|      B|2022-04-17|41200|
|      C|2022-04-11|31500|
|      D|2022-04-12|21500|
|      E|2022-04-13|51000|
+-------+----------+-----+



# UDF 

user defined function:
query 에서 사용하는 사용자 정의 함수

1. 파이썬의 함수로 정의한다
2. spark.udf.register()로 등록한다
3. sql문 안에서 func처럼 사용한다.

In [40]:
from pyspark.sql.types import LongType

#파이썬 함수 만들기
def squared(n):
    return n*n

#함수 등록
spark.udf.register('squared', squared, LongType())

24/12/11 16:18:34 WARN SimpleFunctionRegistry: The function squared replaced a previously registered function.


<function __main__.squared(n)>

In [41]:
df.createOrReplaceTempView("product")

In [42]:
spark.sql('SELECT * FROM product').show()

+-------+----------+-----+
|product|      date|price|
+-------+----------+-----+
|      A|2022-04-16|31200|
|      B|2022-04-17|41200|
|      C|2022-04-11|31500|
|      D|2022-04-12|21500|
|      E|2022-04-13|51000|
+-------+----------+-----+



In [43]:
spark.sql('SELECT price, squared(price) FROM product').show()

+-----+--------------+
|price|squared(price)|
+-----+--------------+
|31200|     973440000|
|41200|    1697440000|
|31500|     992250000|
|21500|     462250000|
|51000|    2601000000|
+-----+--------------+



In [54]:
def read_number(n):
    units = ["","십","백","천","만",]
    nums = '일이삼사오육칠팔구'
    result = []
    i=0
    while n > 0:
        n,r = divmod(n,10)
        if r > 0:
            result.append(nums[r-1]+units[i])
        i += 1
    return "".join(reversed(result))

In [55]:
read_number(123)

'일백이십삼'

In [56]:
spark.udf.register("read_number", read_number)

<function __main__.read_number(n)>

In [57]:
spark.sql('select price, read_number(price) from product').show()

+-----+------------------+
|price|read_number(price)|
+-----+------------------+
|31200|      삼만일천이백|
|41200|      사만일천이백|
|31500|      삼만일천오백|
|21500|      이만일천오백|
|51000|          오만일천|
+-----+------------------+



In [59]:
spark.sql('select price, read_number(price) from product').explain(True)

== Parsed Logical Plan ==
'Project ['price, unresolvedalias('read_number('price), None)]
+- 'UnresolvedRelation [product], [], false

== Analyzed Logical Plan ==
price: bigint, read_number(price): string
Project [price#21L, read_number(price#21L) AS read_number(price)#296]
+- SubqueryAlias product
   +- LogicalRDD [product#19, date#20, price#21L], false

== Optimized Logical Plan ==
Project [price#21L, pythonUDF0#299 AS read_number(price)#296]
+- BatchEvalPython [read_number(price#21L)], [pythonUDF0#299]
   +- Project [price#21L]
      +- LogicalRDD [product#19, date#20, price#21L], false

== Physical Plan ==
*(2) Project [price#21L, pythonUDF0#299 AS read_number(price)#296]
+- BatchEvalPython [read_number(price#21L)], [pythonUDF0#299]
   +- *(1) Project [price#21L]
      +- *(1) Scan ExistingRDD[product#19,date#20,price#21L]



In [61]:
spark.stop()